# Lab | BabyAGI with agent

**Change the planner objective below by changing the objective and the associated prompts and potential tools and agents - Wear your creativity and AI engineering hats
You can't get this wrong!**

You would need the OpenAI API KEY and the [SerpAPI KEY](https://serpapi.com/manage-api-keyhttps://serpapi.com/manage-api-key) to run this lab.


## BabyAGI with Tools

This notebook builds on top of [baby agi](baby_agi.html), but shows how you can swap out the execution chain. The previous execution chain was just an LLM which made stuff up. By swapping it out with an agent that has access to tools, we can hopefully get real reliable information

## Install and Import Required Modules

In [ ]:
!pip install langchain_community langchain_experimental langchain_openai

In [1]:
from typing import Optional

from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_experimental.autonomous_agents import BabyAGI
from langchain_openai import OpenAI, OpenAIEmbeddings

## Connect to the Vector Store

Depending on what vectorstore you use, this step may look different.

In [2]:
# %pip install faiss-cpu > /dev/null
# %pip install google-search-results > /dev/null
from langchain.docstore import InMemoryDocstore
from langchain_community.vectorstores import FAISS

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
SERPAPI_API_KEY = os.getenv('SERPAPI_API_KEY')

In [4]:
# Define your embedding model
embeddings_model = OpenAIEmbeddings()
# Initialize the vectorstore as empty
import faiss

embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


## Define the Chains

BabyAGI relies on three LLM chains:
- Task creation chain to select new tasks to add to the list
- Task prioritization chain to re-prioritize tasks
- Execution Chain to execute the tasks


NOTE: in this notebook, the Execution chain will now be an agent.

In [5]:
from langchain.agents import AgentExecutor, Tool, ZeroShotAgent
from langchain.chains import LLMChain
from langchain_community.utilities import SerpAPIWrapper
from langchain_openai import OpenAI

todo_prompt = PromptTemplate.from_template(
    "You are a planner who is an expert at coming up with a todo list for a given objective. Come up with a todo list for this objective: {objective}"
)
todo_chain = LLMChain(llm=OpenAI(temperature=0), prompt=todo_prompt)
search = SerpAPIWrapper()
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events",
    ),
    Tool(
        name="TODO",
        func=todo_chain.run,
        description="useful for when you need to come up with todo lists. Input: an objective to create a todo list for. Output: a todo list for that objective. Please be very clear what the objective is!",
    ),
]


prefix = """You are an AI who performs one task based on the following objective: {objective}. Take into account these previously completed tasks: {context}."""
suffix = """Question: {task}
{agent_scratchpad}"""
prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["objective", "task", "context", "agent_scratchpad"],
)

C:\Users\jeand\AppData\Local\Temp\ipykernel_40532\842275654.py:9: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  todo_chain = LLMChain(llm=OpenAI(temperature=0), prompt=todo_prompt)


In [6]:
llm = OpenAI(temperature=0)
llm_chain = LLMChain(llm=llm, prompt=prompt)
tool_names = [tool.name for tool in tools]
agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)

C:\Users\jeand\AppData\Local\Temp\ipykernel_40532\3064747660.py:4: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)


### Run the BabyAGI

Now it's time to create the BabyAGI controller and watch it try to accomplish your objective.

In [7]:
OBJECTIVE = "Write a weather report for SF today"

In [8]:
# Logging of LLMChains
verbose = False
# If None, will keep on going forever
max_iterations: Optional[int] = 3
baby_agi = BabyAGI.from_llm(
    llm=llm,
    vectorstore=vectorstore,
    task_execution_chain=agent_executor,
    verbose=verbose,
    max_iterations=max_iterations,
)

In [9]:
baby_agi({"objective": OBJECTIVE})

C:\Users\jeand\AppData\Local\Temp\ipykernel_40532\3867971396.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  baby_agi({"objective": OBJECTIVE})



*****TASK LIST*****

1: Make a todo list

*****NEXT TASK*****

1: Make a todo list


> Entering new AgentExecutor chain...
Thought: I should use the TODO action to create a todo list
Action: TODO
Action Input: "Create a todo list for writing a weather report for SF today"
Observation: 

1. Gather all necessary information: Start by collecting all the necessary information for writing a weather report for San Francisco today. This includes current temperature, humidity, wind speed, and any potential weather events.

2. Check reliable sources: Make sure to check reliable sources such as the National Weather Service or local news stations for accurate and up-to-date information.

3. Determine the format: Decide on the format for the weather report. Will it be a written report, a video, or a combination of both? This will help guide the rest of the process.

4. Create an outline: Before diving into writing, create an outline for the weather report. This will help organize the information 

{'objective': 'Write a weather report for SF today'}

#### Let's try another objective

In [11]:
# Step 1: Reset vectorstore
from langchain.docstore import InMemoryDocstore
from langchain_community.vectorstores import FAISS
import faiss

embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})

# Step 2: Redefine BabyAGI with the same chains
baby_agi = BabyAGI.from_llm(
    llm=llm,
    vectorstore=vectorstore,
    task_execution_chain=agent_executor,
    verbose=verbose,
    max_iterations=max_iterations,
)

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


In [12]:
OBJECTIVE = "Research and summarize the latest advancements in renewable energy technologies"
baby_agi({"objective": OBJECTIVE})



*****TASK LIST*****

1: Make a todo list

*****NEXT TASK*****

1: Make a todo list


> Entering new AgentExecutor chain...
Thought: I should gather information about the latest advancements in renewable energy technologies
Action: Search
Action Input: "latest advancements in renewable energy technologies"
Observation: {'type': 'organic_result'}
Thought: I should filter out irrelevant results and focus on reputable sources
Action: TODO
Action Input: "Create a list of reputable sources for renewable energy advancements"
Observation: 

1. Research and identify reputable sources: Start by researching and identifying reputable sources for renewable energy advancements. This can include government agencies, academic institutions, industry associations, and reputable news outlets.

2. Check for credibility: Once you have a list of potential sources, make sure to check their credibility. Look for any biases or conflicts of interest that may affect the reliability of their information.

3. Gat

{'objective': 'Research and summarize the latest advancements in renewable energy technologies'}

#### Let's try another objective

In [13]:
# Step 1: Reset vectorstore
from langchain.docstore import InMemoryDocstore
from langchain_community.vectorstores import FAISS
import faiss

embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})

# Step 2: Redefine BabyAGI with the same chains
baby_agi = BabyAGI.from_llm(
    llm=llm,
    vectorstore=vectorstore,
    task_execution_chain=agent_executor,
    verbose=verbose,
    max_iterations=max_iterations,
)

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


In [14]:
OBJECTIVE = "I need a program to learn AI and ML from youtube videos"
baby_agi({"objective": OBJECTIVE})


*****TASK LIST*****

1: Make a todo list

*****NEXT TASK*****

1: Make a todo list


> Entering new AgentExecutor chain...
Thought: I need to create a list of tasks to complete
Action: TODO
Action Input: "Create a todo list for learning AI and ML from youtube videos"
Observation: 

1. Research and identify top AI and ML YouTube channels: Start by doing some research to find the best YouTube channels that focus on AI and ML. Look for channels with a large following, high-quality content, and positive reviews from viewers.

2. Subscribe to relevant channels: Once you have identified the top channels, subscribe to them so you can stay updated on their latest videos and content.

3. Create a schedule: Set aside dedicated time each day or week to watch and learn from the YouTube videos. This will help you stay consistent and make progress in your learning.

4. Start with the basics: Begin by watching introductory videos that cover the basics of AI and ML. This will give you a solid foundat

{'objective': 'I need a program to learn AI and ML from youtube videos'}

### BabyAGI with Agent
In this lab, I experimented with BabyAGI powered by LangChain agents and tools, focusing on two different objectives to evaluate how well it can deconstruct abstract goals into meaningful, autonomous task sequences.

#### 🔍 Objective 1: “Research and summarize the latest advancements in renewable energy technologies”
This test was designed to evaluate BabyAGI’s ability to:
- Access live information via the Search tool,
- Create structured workflows using the TODO tool,
- Navigate complex research tasks in a reliable, logical way.

##### ✅ What worked well:
- The agent automatically initiated a high-level planning phase before diving into detailed tasks.
- It used Search to pull recent results (from SerpAPI), and then generated to-do lists covering:
    - Gathering and verifying information
    - Identifying reputable sources
    - Considering future advancements
    - Summarizing data by category (e.g., solar, wind, bioenergy)
- It continued multiple chains with realistic reasoning and returned practical subtasks.

##### 🔁 Challenge handled:
- It briefly attempted to invoke an invalid tool (Check for credibility of sources) but recovered smoothly using valid fallbacks. This reflects solid multi-hop reasoning and robustness in task redirection.

#### 🎓 Objective 2: “I need a program to learn AI and ML from YouTube videos”
This second run tested the agent’s ability to interpret a human-centered learning objective and return a structured self-education plan.

##### ✅ What worked well:
- The agent did not misinterpret “program” as a coding program, but understood it as a learning roadmap.
- It:
    - Identified and listed top 10 AI/ML YouTube channels (Lex Fridman, Sentdex, DeepMind, etc.)
    - Created a schedule for watching and taking notes
    - Recommended online communities and coding practice
    - Proposed advanced follow-ups like building projects, tracking events, and summarizing key takeaways

#### 🧠 Standout qualities:
- The response was highly educational, well-scaffolded, and pedagogically sound.
- It felt like a real personal assistant helping to plan an AI/ML learning journey from scratch.

### 💡 Key Takeaways from Both Runs:
- BabyAGI shines as a task planner and guide, especially when objectives are open-ended or require multi-step research.
- Leveraging Search + TODO allows the agent to mimic how a human might reason through a goal and iterate as needed.
- For optimal performance, it’s important to reset the vectorstore before each new run to prevent memory ID conflicts.
- The system is highly adaptable: it can work with academic, technical, and self-learning goals equally well.

### 🧭 Final Thoughts:
These experiments confirmed that BabyAGI with tools is a powerful prototype for autonomous planning, research summarization, and personalized learning design. It’s especially effective when paired with structured tools and clear objectives, and offers real potential as a backend for intelligent assistants, education tools, or productivity agents.